<a href="https://colab.research.google.com/github/sarthakkaushik/BERT/blob/main/Summarizing_blogs_using_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Transformers and Importing Dependencies

In [1]:
!pip install transformers

     |████████████████████████████████| 3.3 MB 9.9 MB/s 
     |████████████████████████████████| 895 kB 82.7 MB/s 
     |████████████████████████████████| 3.3 MB 50.0 MB/s 
     |████████████████████████████████| 596 kB 53.9 MB/s 
     |████████████████████████████████| 61 kB 89 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

#Load Summarization Pipeline

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [5]:
URL="https://www.sciencenews.org/article/bacteria-virus-phage-shigellosis-arms-race"

In [6]:
r = requests.get(URL)

In [7]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [8]:
ARTICLE

'Support nonprofit journalism. A bacteria-virus arms race could lead to a new way to treat shigellosis When bacteria evolve to become resistant to bacteria-killing viruses known as bacteriophages (illustrated attacking a bacterium), it doesn’t always end well for the bacteria. Kateryna Kon/Science Photo Library/Getty Images Plus By Erin Garcia de Jesús 11 hours ago When some bacteria manage to escape being killed by a virus, the microbes end up hamstringing themselves. And that could be useful in the fight to treat infections. The bacterium Shigella flexneri — one cause of the infectious disease shigellosis — can spread within cells that line the gut by propelling itself through the cells’ barriers. That causes tissue damage that can lead to symptoms like bloody diarrhea. But when S. flexneri in lab dishes evolved to elude a type of bacteria-killing virus, the bacteria couldn’t spread cell to cell anymore, making it less virulent, researchers report November 17 in Applied and Environme

#Chunk Text

In [9]:
max_chunk = 500

In [10]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [11]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [12]:
len(chunks)

2

#Summarize Text

In [13]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [14]:
res[0]

{'summary_text': ' A bacteria-virus arms race could lead to a new way to treat shigellosis . When bacteria evolve to escape being killed by a virus, the microbes end up hamstringing themselves . The research is a hopeful sign for phage therapy .'}

In [15]:
' '.join([summ['summary_text'] for summ in res])

' A bacteria-virus arms race could lead to a new way to treat shigellosis . When bacteria evolve to escape being killed by a virus, the microbes end up hamstringing themselves . The research is a hopeful sign for phage therapy .  Every year, approximately 1. 3 million people die from shigellosis, which is caused by four Shigella species . Phage therapy could be a cheaper, more accessible option to treat the infection .'

In [16]:
text = ' '.join([summ['summary_text'] for summ in res])

In [17]:
text

' A bacteria-virus arms race could lead to a new way to treat shigellosis . When bacteria evolve to escape being killed by a virus, the microbes end up hamstringing themselves . The research is a hopeful sign for phage therapy .  Every year, approximately 1. 3 million people die from shigellosis, which is caused by four Shigella species . Phage therapy could be a cheaper, more accessible option to treat the infection .'